<a href="https://colab.research.google.com/github/sameer-sultan41/langchain-raging-for-e-commerce/blob/main/image_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install pillow --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 11.3.0 which is incompatible.


In [22]:

%pip install facenet-pytorch


In [25]:
import torch
import torch.nn as nn

import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [26]:
from facenet_pytorch import MTCNN, InceptionResnetV1

models = InceptionResnetV1(pretrained = "vggface2").eval()
models

  0%|          | 0.00/107M [00:00<?, ?B/s]

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [27]:
def process_image(image_path):
  image = Image.open(image_path).convert("RGB")
  process = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  ])
  image = process(image).unsqueeze(0)
  return image

def create_image_embading(image_path):
  try:
    input_tensor = process_image(image_path)
    with torch.no_grad():
      embedding = models(input_tensor)
      return embedding.squeeze().numpy()
  except Exception as e:
    print(e)
    return None


In [28]:
!mkdir images


In [29]:
import requests
import os

def save_image_from_url(image_url, image_name):
  try:
    if not os.path.exists("images"):
      os.makedirs("images")

    image_path = os.path.join("images",image_name)
    response = requests.get(image_url, stream=True)
    response.raise_for_status()

    with open(image_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image save to {image_path}")

  except requests.exceptions.RequestException as e:
    print(f"Error to download image {e}")

  except Exception as e:
    print(f"An Error occur {e}")





In [30]:
save_image_from_url("https://scontent.fkhi28-1.fna.fbcdn.net/v/t39.30808-6/508010196_10161196364997623_5064004041179531548_n.jpg?_nc_cat=105&ccb=1-7&_nc_sid=6ee11a&_nc_eui2=AeGUqqTHk8XE5IUyGVmGQaIvGGLiw8otPccYYuLDyi09x6-RuNBpCKisUHygz7fivXVcGh35beFzveXG0LJI4K4t&_nc_ohc=SQwN0HVvNLcQ7kNvwFcOL6G&_nc_oc=AdlU9N4D5g4pAQhleZca8u1qE9o_YaOixPcgboHdXCWaojT0NEzpD5ifArPLn5pbbGo&_nc_zt=23&_nc_ht=scontent.fkhi28-1.fna&_nc_gid=N2lD52XV3_ZpRIVA8ojL5g&oh=00_AfSjRxURG6hY_qmVbNHPtlwyYwEgFRpFqFHvnJNIFa2ndA&oe=68959707","z1")
save_image_from_url("https://scontent.fkhi28-1.fna.fbcdn.net/v/t39.30808-6/515507688_10161292756932623_7294597063831983484_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=a5f93a&_nc_eui2=AeFR2OiPMqpu0IR6nuUhzlRIf676beXdeZ9_rvpt5d15n1SeB7Wf_y-g4JnL2nQ7d8S62uPvKjovNUKONSgyofRh&_nc_ohc=v1XgwiC1euIQ7kNvwF4GOB_&_nc_oc=AdlGc3Qye28iaC3OS0w_bjLvak0U1IWAXCKAijGqZmQLCyZpwqW_GjxYVFcmltoHBZc&_nc_zt=23&_nc_ht=scontent.fkhi28-1.fna&_nc_gid=U07OUSdKarEIiHJpuUWisA&oh=00_AfT6hsnY0LF7Bu1IMD0d0AWSouCS7gj1RDOXrkW1IEu10A&oe=68958E2E","z2")
save_image_from_url("https://scontent.fkhi28-1.fna.fbcdn.net/v/t39.30808-6/481905265_1188180395984388_4514595919978278245_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=cc71e4&_nc_eui2=AeEiHQVy4o4IP5OCIvh_XBPKTtdevjEm7yZO116-MSbvJjKnrmGJz09Hv-Tikp9zgnMUBYcMITI8Yxzl_OuJBiVU&_nc_ohc=ME8RYr6FjLAQ7kNvwEzNaQJ&_nc_oc=AdkUV56ahiPyIIxZ4NT07opAFgb53U8ecGcnEdZF2Wqi1y9V3UGC3rfgvTpn6KQcDJI&_nc_zt=23&_nc_ht=scontent.fkhi28-1.fna&_nc_gid=PY0MeoLN-cZ9VnLdfunwKg&oh=00_AfQUVgjZZh50L9yCPu0ReqzfDySyifRMAO-FrfM6i-PDxA&oe=68958D84","s1")
save_image_from_url("https://scontent.fkhi28-1.fna.fbcdn.net/v/t39.30808-6/480958947_1177589567043471_2063774845508291774_n.jpg?_nc_cat=100&ccb=1-7&_nc_sid=833d8c&_nc_eui2=AeGmbeAAR2J2b060tzqY63_VUS9wsX1-zftRL3CxfX7N-ytAU7REQc7dEgFmDch5Hj7i1qPbG4HMk2p0LLbtB5We&_nc_ohc=Q425PB2itpwQ7kNvwHG-De0&_nc_oc=AdluDPy2Qzujeu4OiD7yD8tdSOyAS3NJFqAQ92I1BHxn0ckGRX_w9FVn1RjJj5KJyqU&_nc_zt=23&_nc_ht=scontent.fkhi28-1.fna&_nc_gid=VnHv2PkQ3nE_TTdyxhR0-g&oh=00_AfTbymi01lO5EQU443b48rIwt4-mVy7ilivejCGvM97nFg&oe=68958D3E","s2")
save_image_from_url("https://i.pinimg.com/736x/c1/ce/19/c1ce196a99b6268a04d2da6365bff516.jpg","h1")
save_image_from_url("https://hamariweb.com/profiles/images/profile/4920-834.jpg","h2")

Image save to images/z1
Image save to images/z2
Image save to images/s1
Image save to images/s2
Image save to images/h1
Image save to images/h2


In [33]:
image_path = "./images/z1"
z1 = create_image_embading(image_path)

print(f"Image : {z1}")

expected np.ndarray (got numpy.ndarray)
Image : None


In [51]:
q1 = create_image_embading("./images/z1")
q2 = create_image_embading("./images/z2")
s1 = create_image_embading("./images/s1")
s2 = create_image_embading("./images/s2")
h1 = create_image_embading("./images/h1")
h2 = create_image_embading("./images/h2")

expected np.ndarray (got numpy.ndarray)
expected np.ndarray (got numpy.ndarray)
expected np.ndarray (got numpy.ndarray)
expected np.ndarray (got numpy.ndarray)
expected np.ndarray (got numpy.ndarray)
expected np.ndarray (got numpy.ndarray)


In [35]:
!pip install -U milvus-lite pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.4/236.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.74.0
    Uninstalling grpcio-1.74.0:
      Successfully uninstalled grpcio-1.74.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


In [43]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension=512  # The vectors we will use in this demo has 384 dimensions
)


In [46]:
data = [
    {"id": 1, "person_name": "qasim", "vector": q1},
    {"id": 2, "person_name": "qasim", "vector": q2},
    {"id": 3, "person_name": "saad", "vector": s1},
    {"id": 4, "person_name": "saad", "vector": s2},
    {"id": 5, "person_name": "hania", "vector": h1},
    {"id": 6, "person_name": "hania", "vector": h2}
]
# Filter out any entries with None vectors
clean_data = [item for item in data if item["vector"] is not None]


In [47]:
res = client.insert(
    collection_name="images",
    data=clean_data
)

In [48]:
save_image_from_url("https://scontent.fkhi28-1.fna.fbcdn.net/v/t39.30808-6/480344594_1175446573924437_9009343456620589306_n.jpg?_nc_cat=111&ccb=1-7&_nc_sid=833d8c&_nc_eui2=AeFcBmwBxBqjgPbSSycLIfLaOTR9F9vr4185NH0X2-vjXxY8Uk_9VjhsF6_a-04X4hDPfSSPd8CvT2K7VC30iJ47&_nc_ohc=18ZS80vdpr0Q7kNvwFe6O6z&_nc_oc=Adm9wiXYI2ICD4t_7IziP9EHracnmGjmZdPCN6CE5n90QV_yqH4Cd1n_Q5J0uK4gHsI&_nc_zt=23&_nc_ht=scontent.fkhi28-1.fna&_nc_gid=tZ1OBIP5oAVPCByj-LGilA&oh=00_AfQsbuvLtUXCHV78dSX92ov4O0Pd1SBVOCRMPzmHPt77rg&oe=6895ADB1","s3")
s3 = create_image_embading("./images/s3")

Image save to images/s3
expected np.ndarray (got numpy.ndarray)


In [50]:
res = client.search(
    collection_name="images",
    data = [s2],
    limit=1,
    output_fields=["id", "person_name"]
)
res

2025-08-04 15:50:26,652 [ERROR][handler]: Unexpected error: [search], No module named 'numpy.strings', <Time: {'RPC start': '2025-08-04 15:50:26.645745', 'Exception': '2025-08-04 15:50:26.652380'}> (decorators.py:158)
2025-08-04 15:50:26,655 [ERROR][search]: Failed to search collection: images (milvus_client.py:411)


MilvusException: <MilvusException: (code=1, message=Unexpected error, message=<No module named 'numpy.strings'>)>